In [16]:
# imports

import tensorflow as tf
import tensorflow.keras as K
from tensorflow.keras.layers import InputLayer, Conv2D, Activation, MaxPooling2D, Dense, Flatten, BatchNormalization
import cv2

print(tf.__version__)

2.7.0


Darknetv1 Model

In [65]:
class YOLOv1(K.models.Sequential):
  def __init__(self):
    super(YOLOv1, self).__init__()
    self.inp_xy = 448
    self.inp_colour_channels = 3


  def leaky_relu_activation(alpha):
    return lambda x: maximum(0.1 * x, x)

  def build_darknet(self):
    # Input Tensor (448,448,3)
    self.add(InputLayer(input_shape = (self.inp_xy,self.inp_xy,self.inp_colour_channels)))
    
    # Conv Layers
    self.add(Conv2D(filters=64, kernel_size= (7, 7), strides=(2,2), padding = 'same'))
    self.add(Activation(activation=K.layers.LeakyReLU(0.1)))
    self.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

    self.add(Conv2D(filters=192, kernel_size= (3, 3), padding = 'same'))
    self.add(Activation(activation=K.layers.LeakyReLU(0.1)))
    self.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

    self.add(Conv2D(filters=128, kernel_size= (1, 1), padding = 'same'))
    self.add(Activation(activation=K.layers.LeakyReLU(0.1)))
    self.add(Conv2D(filters=256, kernel_size= (3, 3), padding = 'same'))
    self.add(Activation(activation=K.layers.LeakyReLU(0.1)))
    self.add(Conv2D(filters=256, kernel_size= (1, 1), padding = 'same'))
    self.add(Activation(activation=K.layers.LeakyReLU(0.1)))
    self.add(Conv2D(filters=512, kernel_size= (3, 3), padding = 'same'))
    self.add(Activation(activation=K.layers.LeakyReLU(0.1)))
    self.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

    # Alternating (3x3) & (1x1) Kernel Conv
    for i in range(0,4):
      self.add(Conv2D(filters=256, kernel_size= (1, 1), padding = 'same'))
      self.add(Activation(activation=K.layers.LeakyReLU(0.1)))
      self.add(Conv2D(filters=512, kernel_size= (3, 3), padding = 'same'))
      self.add(Activation(activation=K.layers.LeakyReLU(0.1)))
    self.add(Conv2D(filters=512, kernel_size= (1, 1), padding = 'same'))
    self.add(Activation(activation=K.layers.LeakyReLU(0.1)))
    self.add(Conv2D(filters=1024, kernel_size= (3, 3), padding = 'same'))
    self.add(Activation(activation=K.layers.LeakyReLU(0.1)))
    self.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

    for i in range(0,2):
      self.add(Conv2D(filters=512, kernel_size= (1, 1), padding = 'same'))
      self.add(Activation(activation=K.layers.LeakyReLU(0.1)))
      self.add(Conv2D(filters=1024, kernel_size= (3, 3), padding = 'same'))
      self.add(Activation(activation=K.layers.LeakyReLU(0.1)))
    
    self.add(Conv2D(filters=1024, kernel_size= (3, 3), padding = 'same'))
    self.add(Activation(activation=K.layers.LeakyReLU(0.1)))
    self.add(Conv2D(filters=1024, kernel_size= (3, 3), strides=(2,2) padding = 'same'))
    self.add(Activation(activation=K.layers.LeakyReLU(0.1)))

    # Fully Connected Layer
    self.add(Flatten())
    self.add(Dense(512))
    self.add(Dense(1024))
    self.add(Dropout(0.5))
    self.add()


    

In [64]:
# Create TF Sequential
model = YOLOv1()

# Create Darknet Arch
model.build_darknet()

# Display Model
model.summary()

xd
xd
xd
xd
Model: "yol_ov1_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 224, 224, 64)      9472      
                                                                 
 activation_18 (Activation)  (None, 224, 224, 64)      0         
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 112, 112, 64)     0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (None, 112, 112, 192)     110784    
                                                                 
 activation_19 (Activation)  (None, 112, 112, 192)     0         
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 56, 56, 192)      0         
 g2D)                                       